<a href="https://colab.research.google.com/github/Saaaaangmin/LLM/blob/main/LLM_LogAnal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re

# 로그 한 줄을 나타내는 정규 표현식
log_pattern = re.compile(
    r"^\[(?P<발생시간>\d{8} \d{2}:\d{2}:\d{2}(?: \d{3})?)\] "  # 발생 시간
    r"\[(?P<Port>\d{6})\] "                                # Com Port
    r"\[(?P<유형>[^\]]+)\] "                                # 유형 또는 함수명
    r"\[(?P<에러메시지>.*)\]$"                               # 메시지
)

# 에러 메시지 매핑 (에러 키워드와 설명을 연결)
error_descriptions = {
    "-1 port open error!": "-1 Port open error = 시리얼의 Com Port가 오픈되지 않은 상태로, 케이블 상태 및 COM Port/Baudrate 값이 올바른지 확인이 필요합니다.",
    "-3 Ack error!": "-3 Ack Error = 시리얼 데이터 ACK 수신 오류이므로, 시리얼 연동에 문제가 없는지 확인이 필요합니다.",
    "-4 lrc, cancel!": "-4 lrc, cancel = 시리얼 통신 중 LRC 체크 에러가 발생하여 응답 데이터를 수신하지 못한 상황으로, 시리얼 통신이 정상적으로 되고 있는지 확인이 필요합니다.",
    "Timeout": "Timeout Error = 응답 시간이 초과되었습니다. 연결 상태를 확인하세요.",
}

In [2]:
# 추적할 함수명 목록
function_names = ["ReqToCat", "ReqStop", "SetBmpFile", "SetLogDir", "GetDecSignData", "GetDllVer"]

def parse_log_line(line):
    """로그의 한 줄을 분석하여 정보를 추출"""
    match = log_pattern.match(line)
    if match:
        return match.groupdict()  # dictionary 형태로 반환
    else:
        return None

def analyze_log_file(file_path):
    """파일의 로그를 한 줄씩 읽어 분석하고 ERROR 메시지를 수집"""
    error_logs = []  # ERROR 메시지를 저장할 리스트
    last_function_name = "Unknown"  # 직전 줄의 함수명을 저장

    try:
        with open(file_path, 'r', encoding='cp949') as file:
            for line in file:
                parsed_line = parse_log_line(line.strip())

                if parsed_line:
                    # function_names 목록에 있는 함수명이 감지될 때 업데이트
                    if parsed_line['유형'].strip() in function_names:
                        last_function_name = parsed_line['유형'].strip()

                    # ERROR 메시지를 찾을 경우
                    if "ERROR" in parsed_line['유형']:
                        # 에러 메시지에 설명 추가
                        error_message = parsed_line['에러메시지']
                        detailed_message = error_descriptions.get(error_message, error_message)

                        # 딕셔너리 형태로 필요한 항목 저장
                        error_logs.append({
                            "발생시간": parsed_line['발생시간'],
                            "Port": parsed_line['Port'],
                            "함수명": last_function_name,  # 추적된 함수명 사용
                            "에러메시지": detailed_message
                        })

        # 수집된 ERROR 메시지 출력
        print("ERROR 메시지 수집 결과:")
        for error in error_logs:
            print(f"발생시간: {error['발생시간']}")
            print(f"Port: {error['Port']}")
            print(f"함수명: {error['함수명']}")
            print(f"에러메시지: {error['에러메시지']}")
            print('-' * 200)  # 구분선
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
    except PermissionError:
        print(f"파일에 접근할 권한이 없습니다: {file_path}")
    except UnicodeDecodeError as e:
        print(f"파일을 읽는 중 인코딩 오류가 발생했습니다: {e}")
    except Exception as e:
        print(f"예상치 못한 오류가 발생했습니다: {e}")

# 테스트 실행 (로그 파일 경로를 지정)
log_file_path = r'C:\NICElog\20240822.log'
analyze_log_file(log_file_path)

파일을 찾을 수 없습니다: C:\NICElog\20240822.log
